# Установка библиотеки

In [1]:
!pip install pillow_heif

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.4 MB 5.1 MB/s 


In [2]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 80 kB/s 


# Начало

In [3]:
import pandas as pd
import os
from PIL import Image, ImageDraw, ImageFilter
import numpy as np
import pillow_heif
from tqdm import tqdm
import cv2
from matplotlib import pyplot as plt
import matplotlib
import imutils
import json
import pickle
import torch
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from functools import lru_cache
from catboost import CatBoostRegressor
import shutil

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
WORKING_DIR = ''#'/content/drive/MyDrive/ai/cardistance/'

In [6]:
test_img_names = list(set(os.listdir(WORKING_DIR + 'datasets/test')))
train_img_names = list(set(os.listdir(WORKING_DIR + 'datasets/train')))

In [7]:
train_labels_first_df = pd.read_csv(WORKING_DIR + 'datasets/train.csv', sep=';', index_col=None)
train_labels_first_df = train_labels_first_df.rename({'image_name': 'img_name'}, axis='columns')

In [8]:
model_vs = ['v5n', 'v5s', 'v5m', 'v5l', 'v5x', 'v5n6', 'v5s6', 'v5m6', 'v5l6', 'v5x6']

In [9]:
DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [10]:
MODEL_VERSIONS_PRIORITY = ['v5l6', 'v5n', 'v5n6']
W_IMG, H_IMG = 4032, 3024

In [11]:
DATA_TRAINING_COLS = ['xmin_car', 'ymin_car', 'xmax_car', 'ymax_car', 'xmin_plate', 'ymin_plate', 'xmax_plate', 'ymax_plate', 'distance']
DATA_TESTING_COLS = DATA_TRAINING_COLS[:-1]

In [12]:
@lru_cache(maxsize=128)
def get_img(img_name):
    if img_name in train_img_names:
        dataset_dir = WORKING_DIR + 'datasets/train/'
    else:
        dataset_dir = WORKING_DIR + 'datasets/test/'

    if 'heic' in img_name:
        heif_file = pillow_heif.read_heif(os.path.join(dataset_dir, img_name))
        img = Image.frombytes(
            heif_file.mode,
            heif_file.size,
            heif_file.data,
            "raw"
        )
    else:
        img = Image.open(os.path.join(dataset_dir, img_name))
    
    return img

# Анализ

In [ ]:
# проверка размеров картинок
sizes = set()
for img_name in tqdm(list(train_img_names) + list(test_img_names)):
    if img_name in train_img_names:
        dataset_dir = WORKING_DIR + 'datasets/train/'
    else:
        dataset_dir = WORKING_DIR + 'datasets/test/'

    if 'heic' in img_name:
        heif_file = pillow_heif.read_heif(os.path.join(dataset_dir, img_name))
        img = Image.frombytes(
            heif_file.mode,
            heif_file.size,
            heif_file.data,
            "raw"
        )
    else:
        img = Image.open(os.path.join(dataset_dir, img_name))
    
    sizes.add(img.size)

In [19]:
sizes

{(4032, 3024)}

#  Train/Test

In [20]:
# вырезание номерных знаков
cascade_classifier = cv2.CascadeClassifier(WORKING_DIR + 'haarcascade_russian_plate_number.xml')
def get_car_plate(img, x_shift=0, y_shift=0):
    img = np.array(img)
    img = img[:, :, ::-1].copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # convert to grey scale
    detected = cascade_classifier.detectMultiScale(gray, 1.2)

    new_imgs = []
    for numbers in detected:
        x, y, w, h = numbers
        x += x_shift
        y += y_shift
        new_imgs.append([x, y, w, h])
    return new_imgs

In [24]:
def predict_by_yolov_model(model_v):
    if not os.path.exists(WORKING_DIR + 'yolo_data'):
        os.makedirs(WORKING_DIR + 'yolo_data')

    model = torch.hub.load('ultralytics/yolov5', f'yolo{model_v}')
    model.classes = [2]
    model.to(DEVICE)

    # вырезание машин + номерных знаков предыдущим методом
    train_data_cars = []
    train_data_plates = []

    for img_name in tqdm(list(sorted(train_img_names))): 
        img = get_img(img_name)
        w, h = img.size
        results = model(np.array(img)).xyxy[0]
        centerized_results = [(result[0] + (result[2] - result[0]) / 2, 
                               result[1] + (result[3] - result[1]) / 2,
                               i)
                              for i, result in enumerate(results)]
        if not any(centerized_results):
            print(img_name)
            continue
        
        centerized_results = list(sorted(centerized_results, key=lambda x: ((w / 2 - x[0]) ** 2 + (h / 2 - x[1]) ** 2) ** 0.5))
        for centerized_result in centerized_results:
            center_result = results[centerized_result[-1]]
            result = list(map(int, center_result))[:4]
            car_result = [[result[0], result[1], result[2] - result[0], result[3] - result[1]]]
            croped_img = img.crop(result)
            plate_imgs = get_car_plate(croped_img, result[0], result[1])
            if any(plate_imgs):
                break
        
        if not any(plate_imgs):
            print(img_name)
            #continue

        train_data_cars.append([img_name] + car_result)
        train_data_plates.append([img_name] + plate_imgs)

        with open(WORKING_DIR + f'yolo_data/train_plates_{model_v}.pkl', 'wb') as f:
            pickle.dump(train_data_plates, f)

        with open(WORKING_DIR + f'yolo_data/train_cars_{model_v}.pkl', 'wb') as f:
            pickle.dump(train_data_cars, f)
        
    # вырезание машин + номерных знаков предыдущим методом
    test_data_cars = []
    test_data_plates = []

    for img_name in tqdm(list(sorted(test_img_names))): 
        img = get_img(img_name)
        w, h = img.size
        results = model(np.array(img)).xyxy[0]
        centerized_results = [(result[0] + (result[2] - result[0]) / 2, 
                               result[1] + (result[3] - result[1]) / 2,
                               i)
                              for i, result in enumerate(results)]
        if not any(centerized_results):
            print(img_name)
            continue

        centerized_results = list(sorted(centerized_results, key=lambda x: ((w / 2 - x[0]) ** 2 + (h / 2 - x[1]) ** 2) ** 0.5))
        for centerized_result in centerized_results:
            center_result = results[centerized_result[-1]]
            result = list(map(int, center_result))[:4]
            car_result = [[result[0], result[1], result[2] - result[0], result[3] - result[1]]]
            croped_img = img.crop(result)
            plate_imgs = get_car_plate(croped_img, result[0], result[1])
            if any(plate_imgs):
                break
        
        if not any(plate_imgs):
            print(img_name)
            #continue

        test_data_cars.append([img_name] + car_result)
        test_data_plates.append([img_name] + plate_imgs)

        with open(WORKING_DIR + f'yolo_data/test_plates_{model_v}.pkl', 'wb') as f:
            pickle.dump(test_data_plates, f)

        with open(WORKING_DIR + f'yolo_data/test_cars_{model_v}.pkl', 'wb') as f:
            pickle.dump(test_data_cars, f)

In [ ]:
for model_v in model_vs:
    print('-' * 20)
    print(model_v)
    predict_by_yolov_model(model_v)

# Init

In [13]:
train_datas = {}
test_datas = {}

In [14]:
for model_v in model_vs:
    train_datas[model_v] = {}
    test_datas[model_v] = {}
    for data_type in ['cars', 'plates']:
        train_data = pickle.load(open(WORKING_DIR + f'yolo_data/train_{data_type}_{model_v}.pkl', 'rb'))
        test_data = pickle.load(open(WORKING_DIR + f'yolo_data/test_{data_type}_{model_v}.pkl', 'rb'))

        train_data = {annotation[0]: annotation[1:] for annotation in train_data}
        test_data = {annotation[0]: annotation[1:] for annotation in test_data}

        train_datas[model_v][data_type] = train_data
        test_datas[model_v][data_type] = test_data

# Работа с данными

In [15]:
# нормализирует данные номеров машин
def get_normalized_plates_data(plates_data, cars_data):
    data_plates_normalized = {img_name:
                              [[(annotation[0] - cars_data[img_name][0][0]) / cars_data[img_name][0][2], 
                                (annotation[1] - cars_data[img_name][0][1]) / cars_data[img_name][0][3],
                                annotation[2] / cars_data[img_name][0][2],
                                annotation[3] / cars_data[img_name][0][3]]
                               for annotation in annotations]
                              for img_name, annotations in plates_data.items()}
    return data_plates_normalized

In [16]:
# выбирает индексы самых "центральных" номеров машин
def get_center_plate_indexes(plates_data, plate_mid):
    plate_indexes = {}
    for img_name, annotations in plates_data.items():
        centers = [[annotation[0] + annotation[2] / 2, annotation[1] + annotation[3] / 2] for annotation in annotations]
        distances = [((center[0] - plate_mid[0]) ** 2 + (center[1] - plate_mid[1]) ** 2) ** 0.5 for center in centers]
        plate_index = distances.index(min(distances))
        plate_indexes[img_name] = plate_index
    return plate_indexes

In [17]:
# выбирает номера машин по индексам
def get_data_plates_from_choose(data_plates, data_plates_choose):
    new_data_plates = {}
    for img_name, annotations in data_plates.items():
        if len(annotations) == 0:
            adding = []
        elif len(annotations) == 1:
            adding = [annotations[0]]
        else:
            adding = [annotations[data_plates_choose[img_name]]]
        new_data_plates[img_name] = adding
    return new_data_plates

In [18]:
# выбор центральных номеров машин
def get_deleted_noncenter_plates(train_data_cars, test_data_cars, train_data_plates, test_data_plates):
    train_data_plates_normalized = get_normalized_plates_data(train_data_plates, train_data_cars)
    test_data_plates_normalized = get_normalized_plates_data(test_data_plates, test_data_cars)
    
    mid_coords = [[annotations[0][0] + annotations[0][2] / 2, annotations[0][1] + annotations[0][3] / 2]
                  for annotations in list(train_data_plates_normalized.values()) + list(test_data_plates_normalized.values())
                  if len(annotations) == 1]
    plate_mid = [sum(map(lambda x: x[0], mid_coords)) / len(mid_coords), sum(map(lambda x: x[1], mid_coords)) / len(mid_coords)]

    train_data_plates_choose = {img_name: annotations for img_name, annotations in train_data_plates_normalized.items() if len(annotations) > 1}
    test_data_plates_choose = {img_name: annotations for img_name, annotations in test_data_plates_normalized.items() if len(annotations) > 1}

    train_data_plates_choose = get_center_plate_indexes(train_data_plates_choose, plate_mid)
    test_data_plates_choose = get_center_plate_indexes(test_data_plates_choose, plate_mid)

    train_data_plates = get_data_plates_from_choose(train_data_plates, train_data_plates_choose)
    test_data_plates = get_data_plates_from_choose(test_data_plates, test_data_plates_choose)

    return train_data_plates, test_data_plates

In [19]:
# соединить данные об аннотациях машин и номеров 
def combine_cars_plates_data(data_cars, data_plates, img_names):
    img_names_plates = list(filter(lambda x: x in data_cars.keys() and x in data_plates.keys(), img_names))

    data = []
    for img_name in img_names:
        adding = [img_name]
        if img_name in data_cars.keys():
            adding += data_cars[img_name]
            if img_name in data_plates.keys():
                adding += data_plates[img_name]
        data.append(adding)

    return data

In [20]:
# расширение данных
def extend_data_df(data_df):
    data_df['w_img'] = W_IMG
    data_df['h_img'] = H_IMG
    
    for suffix1 in ['_car', '_plate']:
        data_df['xmax' + suffix1] = data_df['xmin' + suffix1] + data_df['w' + suffix1]
        data_df['ymax' + suffix1] = data_df['ymin' + suffix1] + data_df['h' + suffix1]
    
    for suffix1 in ['_car', '_plate']:
        data_df['distance_to_center' + suffix1] = (((data_df['xmin_car'] + data_df['w_car'] / 2) - data_df['w_img'] / 2) ** 2 + 
                                                   ((data_df['ymin_car'] + data_df['h_car'] / 2) - data_df['h_img'] / 2) ** 2) ** 0.5

In [21]:
for model_v in model_vs:
    train_data_cars = train_datas[model_v]['cars']
    train_data_plates = train_datas[model_v]['plates']
    test_data_cars = test_datas[model_v]['cars']
    test_data_plates = test_datas[model_v]['plates']

    train_data_plates, test_data_plates = get_deleted_noncenter_plates(train_data_cars, test_data_cars, train_data_plates, test_data_plates)

    # выбор первого элемента у всех аннотаций
    select_first = lambda data: {key: value[0] for key, value in data.items() if any(value)}

    train_data_cars = select_first(train_data_cars)
    train_data_plates = select_first(train_data_plates)
    test_data_cars = select_first(test_data_cars)
    test_data_plates = select_first(test_data_plates)

    train_data = combine_cars_plates_data(train_data_cars, train_data_plates, train_img_names)
    test_data = combine_cars_plates_data(test_data_cars, test_data_plates, test_img_names)
    
    train_data_df = pd.DataFrame(train_data, columns=['img_name', 'xmin_car', 'ymin_car', 'w_car', 'h_car', 'xmin_plate', 'ymin_plate', 'w_plate', 'h_plate'])
    test_data_df = pd.DataFrame(test_data, columns=['img_name', 'xmin_car', 'ymin_car', 'w_car', 'h_car', 'xmin_plate', 'ymin_plate', 'w_plate', 'h_plate'])

    train_data_df = pd.concat([train_data_df, test_data_df])
    train_data_df = pd.merge(train_labels_first_df, train_data_df, how='left')

    extend_data_df(train_data_df)
    extend_data_df(test_data_df)

    # очищение данных, где не выделены номерные знаки
    train_data_df = train_data_df[train_data_df['xmin_plate'].notna()]
    test_data_df = test_data_df[test_data_df['xmin_plate'].notna()]

    # очищение аннотаций машин, не близких к центру
    train_data_df = train_data_df[train_data_df['distance_to_center_car'] < 500]
    test_data_df = test_data_df[test_data_df['distance_to_center_car'] < 500]

    train_datas[model_v] = train_data_df
    test_datas[model_v] = test_data_df

# Обучение моделей

In [22]:
# обучение модели на датасете
def get_trained_model(train_data_df):
    train_data_df = train_data_df.dropna()
    model = CatBoostRegressor(verbose=False)
    model.fit(train_data_df.drop(columns=['distance']), train_data_df['distance'])
    return model

In [23]:
models = {}
for model_v in model_vs:
    data_df = train_datas[model_v].copy()

    models[model_v] = []
    models[model_v].append(get_trained_model(data_df[DATA_TRAINING_COLS]))

# Сохранение моделей

In [24]:
saving_path = WORKING_DIR + 'catboosting_models/out/first/'
for model_type, cb_models in models.items():
    saving_models_path = saving_path + model_type + '/'
    if not os.path.exists(saving_models_path):
        os.makedirs(saving_models_path)
    for i, model in enumerate(cb_models):
        if isinstance(model, CatBoostRegressor):
            model.save_model(saving_models_path + str(i) + '.cb')
        else:
            pickle.dump(model, open(saving_models_path + str(i) + '.pkl', 'wb'))

# Загрузка моделей

In [25]:
models = {}
loading_path = WORKING_DIR + 'catboosting_models/out/first/'
for model_type in os.listdir(loading_path):
    loading_models_path = loading_path + model_type + '/'
    models[model_type] = []
    for cb_model_name in sorted(os.listdir(loading_models_path), key=lambda x: int(x.replace('.cb', ''))):
        loading_model_path = loading_models_path + cb_model_name
        model = CatBoostRegressor()
        model.load_model(loading_model_path)
        models[model_type].append(model)

# Предсказания

In [26]:
# получение предсказание модели
def get_prediction(data_df, img_names, model):
    data_df = data_df.dropna()

    preds = model.predict(data_df.drop(columns=['img_name']))
    data_df['distance'] = preds

    data_predicted = data_df[['img_name', 'distance']]

    lost_test_items = []
    for img_name in set(img_names) - set(data_predicted['img_name'].values):
        distance = 0
        lost_test_items.append([img_name, distance])
    lost_test_items_df = pd.DataFrame(lost_test_items, columns=['img_name', 'distance'])

    data_predicted = pd.concat([data_predicted, lost_test_items_df])
    data_predicted = data_predicted.sort_values(by='img_name')

    return data_predicted

In [27]:
all_test_preds = {}
for model_v in model_vs:
    test_data_df = test_datas[model_v].copy()
    all_test_preds[model_v] = []

    model = models[model_v][0]
    data_predicted = get_prediction(test_data_df[DATA_TESTING_COLS + ['img_name']], test_img_names, model)
    all_test_preds[model_v].append(data_predicted)

# Обработка предсказаний all_test_preds

In [28]:
submission = {}
for model_v in model_vs:
    preds = all_test_preds[model_v][0]
    for img_name in preds['img_name']:
        distance = preds[preds['img_name'] == img_name]['distance'].values[0]
        if distance != 0:
            submission[img_name] = submission.get(img_name, []) + [distance]

In [29]:
submission_df = pd.DataFrame({}, columns=['img_name', 'distance'])
for img_name, distances in submission.items():
    row = {'img_name': img_name, 'distance': sum(distances) / len(distances)}
    submission_df = submission_df.append(row, ignore_index=True)

In [30]:
test_labels_df = submission_df.sort_values(by='img_name').reset_index()
del test_labels_df['index']

In [31]:
test_labels_df

,img_name,distance
0,img_1597.jpg,3.041297
1,img_1598.jpg,2.282192
2,img_1599.jpg,1.722570
3,img_1602.jpg,2.351755
4,img_1604.jpg,1.682044
...,...,...
516,img_2936.jpg,2.851154
517,img_2938.jpg,2.457940
518,img_2942.jpg,4.026504
519,img_2943.jpg,4.255964


# Далее

In [32]:
if not os.path.exists(WORKING_DIR + 'submit'):
    os.makedirs(WORKING_DIR + 'submit')

In [33]:
for iteration in tqdm(range(48)):
    train_labels_df = pd.concat([train_labels_first_df, test_labels_df])
    train_datas = {}
    test_datas = {}
    for model_v in model_vs:
        train_datas[model_v] = {}
        test_datas[model_v] = {}
        for data_type in ['cars', 'plates']:
            train_data = pickle.load(open(WORKING_DIR + f'yolo_data/train_{data_type}_{model_v}.pkl', 'rb'))
            test_data = pickle.load(open(WORKING_DIR + f'yolo_data/test_{data_type}_{model_v}.pkl', 'rb'))

            train_data = {annotation[0]: annotation[1:] for annotation in train_data}
            test_data = {annotation[0]: annotation[1:] for annotation in test_data}

            train_datas[model_v][data_type] = train_data
            test_datas[model_v][data_type] = test_data
    for model_v in model_vs:
        train_data_cars = train_datas[model_v]['cars']
        train_data_plates = train_datas[model_v]['plates']
        test_data_cars = test_datas[model_v]['cars']
        test_data_plates = test_datas[model_v]['plates']

        train_data_plates, test_data_plates = get_deleted_noncenter_plates(train_data_cars, test_data_cars, train_data_plates, test_data_plates)

        # выбор первого элемента у всех аннотаций
        select_first = lambda data: {key: value[0] for key, value in data.items() if any(value)}

        train_data_cars = select_first(train_data_cars)
        train_data_plates = select_first(train_data_plates)
        test_data_cars = select_first(test_data_cars)
        test_data_plates = select_first(test_data_plates)

        train_data = combine_cars_plates_data(train_data_cars, train_data_plates, train_img_names)
        test_data = combine_cars_plates_data(test_data_cars, test_data_plates, test_img_names)

        train_data_df = pd.DataFrame(train_data, columns=['img_name', 'xmin_car', 'ymin_car', 'w_car', 'h_car', 'xmin_plate', 'ymin_plate', 'w_plate', 'h_plate'])
        test_data_df = pd.DataFrame(test_data, columns=['img_name', 'xmin_car', 'ymin_car', 'w_car', 'h_car', 'xmin_plate', 'ymin_plate', 'w_plate', 'h_plate'])

        train_data_df = pd.concat([train_data_df, test_data_df])
        ###train_labels_df/train_labels_first_df
        train_data_df = pd.merge(train_labels_df, train_data_df, how='left')
        ###

        extend_data_df(train_data_df)
        extend_data_df(test_data_df)

        # очищение данных, где не выделены номерные знаки
        train_data_df = train_data_df[train_data_df['xmin_plate'].notna()]
        test_data_df = test_data_df[test_data_df['xmin_plate'].notna()]

        # очищение аннотаций машин, не близких к центру
        train_data_df = train_data_df[train_data_df['distance_to_center_car'] < 500]
        test_data_df = test_data_df[test_data_df['distance_to_center_car'] < 500]

        train_datas[model_v] = train_data_df
        test_datas[model_v] = test_data_df

    models = {}
    for model_v in model_vs:
        data_df = train_datas[model_v].copy()

        models[model_v] = []
        models[model_v].append(get_trained_model(data_df[DATA_TRAINING_COLS]))

    saving_path = WORKING_DIR + f'catboosting_models/out/{iteration}/'
    for model_type, cb_models in models.items():
        saving_models_path = saving_path + model_type + '/'
        if not os.path.exists(saving_models_path):
            os.makedirs(saving_models_path)
        for i, model in enumerate(cb_models):
            if isinstance(model, CatBoostRegressor):
                model.save_model(saving_models_path + str(i) + '.cb')
            else:
                pickle.dump(model, open(saving_models_path + str(i) + '.pkl', 'wb'))
    
    models = {}
    loading_path = saving_path
    for model_type in os.listdir(loading_path):
        loading_models_path = loading_path + model_type + '/'
        models[model_type] = []
        for cb_model_name in sorted(os.listdir(loading_models_path), key=lambda x: int(x.replace('.cb', ''))):
            loading_model_path = loading_models_path + cb_model_name
            model = CatBoostRegressor()
            model.load_model(loading_model_path)
            models[model_type].append(model)
    
    all_test_preds = {}
    for model_v in model_vs:
        test_data_df = test_datas[model_v].copy()
        all_test_preds[model_v] = []

        model = models[model_v][0]
        data_predicted = get_prediction(test_data_df[DATA_TESTING_COLS + ['img_name']], test_img_names, model)
        all_test_preds[model_v].append(data_predicted)
    
    submission = {}
    for model_v in model_vs:
        preds = all_test_preds[model_v][0]
        for img_name in preds['img_name']:
            distance = preds[preds['img_name'] == img_name]['distance'].values[0]
            if distance != 0:
                submission[img_name] = submission.get(img_name, []) + [distance]
    
    submission_df = pd.DataFrame({}, columns=['img_name', 'distance'])
    for img_name, distances in submission.items():
        row = {'img_name': img_name, 'distance': sum(distances) / len(distances)}
        submission_df = submission_df.append(row, ignore_index=True)
    
    test_labels_df = submission_df.sort_values(by='img_name').reset_index()
    del test_labels_df['index']
    test_labels_df.to_csv(WORKING_DIR + f'submit/catboosting_train_on_full_data_all_models_{iteration + 1}.csv', sep=';', index=False)

100%|██████████| 48/48 [25:28<00:00, 31.84s/it]


In [34]:
test_labels_df

,img_name,distance
0,img_1597.jpg,2.962202
1,img_1598.jpg,2.156337
2,img_1599.jpg,1.707230
3,img_1602.jpg,2.274215
4,img_1604.jpg,1.433453
...,...,...
516,img_2936.jpg,2.554438
517,img_2938.jpg,1.815709
518,img_2942.jpg,4.066003
519,img_2943.jpg,4.158153


In [35]:
test_labels_df.round(2).to_csv(f'out.csv', sep=';', index=False)